In [ ]:
cd ../..

In [ ]:
import json
import pandas as pd
import torch
from src.utils import augment_questions
from src.HF_galois import HF_SPWJ_seq,run_question


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
final_sample_df = pd.read_csv('data/Final_Queries.csv')

In [ ]:
from src.QueryTree import Node

join_query_trees={}
# 40
# Which language is the most popular in Aruba?
q = 'SELECT T2.Language FROM country AS T1 JOIN countrylanguage AS T2 ON T1.Code = T2.CountryCode WHERE T1.Name = "Aruba" ORDER BY Percentage DESC LIMIT 1'
# ┌───────────────────────────┐                             
# │         PROJECTION        │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │           #[0.0]          │                             
# └─────────────┬─────────────┘                                                          
# ┌─────────────┴─────────────┐                             
# │           LIMIT           │                             
# └─────────────┬─────────────┘                                                          
# ┌─────────────┴─────────────┐                             
# │          ORDER_BY         │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │          ORDERS:          │                             
# │           #[0.1]          │                             
# └─────────────┬─────────────┘                                                          
# ┌─────────────┴─────────────┐                             
# │         PROJECTION        │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │          Language         │                             
# │         Percentage        │                             
# └─────────────┬─────────────┘                                                          
# ┌─────────────┴─────────────┐                             
# │           FILTER          │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │      (Name = 'Aruba')     │                             
# └─────────────┬─────────────┘                                                          
# ┌─────────────┴─────────────┐                             
# │      COMPARISON_JOIN      │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │           INNER           ├──────────────┐              
# │    (Code = CountryCode)   │              │              
# └─────────────┬─────────────┘              │                                           
# ┌─────────────┴─────────────┐┌─────────────┴─────────────┐
# │          SEQ_SCAN         ││          SEQ_SCAN         │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   ││   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │          country          ││      countrylanguage      │
# └───────────────────────────┘└───────────────────────────┘                             

# NOT PARSED
c1 = Node()
c1.text = ['SEQ_SCAN','country']
c1.op = 'SEQ_SCAN'
c1.args=['country']

c2 = Node()
c2.text = ['SEQ_SCAN','countrylanguage']
c2.op = 'SEQ_SCAN'
c2.args=['countrylanguage']

c3 = Node()
c3.text = ['JOIN','city']
c3.op = 'JOIN'
c3.key_left='What is the country code of !!x!!?. Answer briefly.'
c3.key_right = 'What is the country code of the country that speaks !!x!!? Answer briefly.'
c3.filter_key='left'

c4 = Node()
c4.text = ['FILTER',"(Name = 'Aruba')"]
c4.op = 'FILTER'
c4.args=["(Name = 'Aruba')"]

c5 = Node()
c5.text = ['PROJECTION','Language','Percentage']
c5.op = 'PROJECTION'
c5.args=['Language','Percentage']


c6 = Node()
c6.text = ['ORDER_BY','Percentage']
c6.op = 'ORDER_BY'
c6.args=['Percentage']


c7 = Node()
c7.text = ['PROJECTION','Language']
c7.op = 'PROJECTION'
c7.args=['Language']

c3.l=c1
c3.r = c2
c4.l=c3
c5.l=c4
c6.l=c5
c7.l=c6






# =======================================
# 41
# what is the capital of states that have cities named durham
q = 'SELECT t2.capital FROM state AS t2 JOIN city AS t1 ON t2.state_name = t1.state_name WHERE t1.city_name = "durham";'
# ┌───────────────────────────┐                             
# │         PROJECTION        │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │          capital          │                             
# └─────────────┬─────────────┘                                                          
# ┌─────────────┴─────────────┐                             
# │           FILTER          │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │   (city_name = 'durham')  │                             
# └─────────────┬─────────────┘                                                          
# ┌─────────────┴─────────────┐                             
# │      COMPARISON_JOIN      │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │           INNER           ├──────────────┐              
# │ (state_name = state_name) │              │              
# └─────────────┬─────────────┘              │                                           
# ┌─────────────┴─────────────┐┌─────────────┴─────────────┐
# │          SEQ_SCAN         ││          SEQ_SCAN         │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   ││   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │           state           ││            city           │
# └───────────────────────────┘└───────────────────────────┘                             

c1 = Node()
c1.text = ['SEQ_SCAN','state']
c1.op = 'SEQ_SCAN'
c1.args=['state']

c2 = Node()
c2.text = ['SEQ_SCAN','city']
c2.op = 'SEQ_SCAN'
c2.args=['city']

c3 = Node()
c3.text = ['JOIN','city']
c3.op = 'JOIN'
c3.key_left='What is the state name of !!x!!? Answer briefly.'
c3.key_right = 'What is the state name of !!x!!? Answer briefly.'
c3.filter_key='right'

c4 = Node()
c4.text = ['FILTER',"(city_name = 'durham')"]
c4.op = 'FILTER'
c4.args=["(city_name = 'durham')"]
c4.filled_question = 'Is !!x!! the same as Durham?'
c5 = Node()
c5.text = ['PROJECTION','capital']
c5.op = 'PROJECTION'
c5.args=['capital']
c5.filled_question = 'What is the capital of state of !!x!!?'


c3.l=c1
c3.r = c2
c4.l=c3
c5.l=c4

join_query_trees[q] = c5



# NOT PARSED
# =======================================
# 42
# What are the regions that use English or Dutch?
q = 'SELECT DISTINCT T1.Region FROM country AS T1 JOIN countrylanguage AS T2 ON T1.Code = T2.CountryCode WHERE T2.Language = "English" OR T2.Language = "Dutch"'
# ┌───────────────────────────┐                             
# │          DISTINCT         │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │       "T1"."Region"       │                             
# └─────────────┬─────────────┘                                                          
# ┌─────────────┴─────────────┐                             
# │         PROJECTION        │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │           Region          │                             
# └─────────────┬─────────────┘                                                          
# ┌─────────────┴─────────────┐                             
# │           FILTER          │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │((Language = 'English') OR │                             
# │   (Language = 'Dutch'))   │                             
# └─────────────┬─────────────┘                                                          
# ┌─────────────┴─────────────┐                             
# │      COMPARISON_JOIN      │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │           INNER           ├──────────────┐              
# │    (Code = CountryCode)   │              │              
# └─────────────┬─────────────┘              │                                           
# ┌─────────────┴─────────────┐┌─────────────┴─────────────┐
# │          SEQ_SCAN         ││          SEQ_SCAN         │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   ││   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │          country          ││      countrylanguage      │
# └───────────────────────────┘└───────────────────────────┘                             

c1 = Node()
c1.text = ['SEQ_SCAN','country']
c1.op = 'SEQ_SCAN'
c1.args=['country']

c2 = Node()
c2.text = ['SEQ_SCAN','countrylanguage']
c2.op = 'SEQ_SCAN'
c2.args=['countrylanguage']

c3 = Node()
c3.text = ['JOIN','city']
c3.op = 'JOIN'
c3.key_left='What is the country code of !!x!!? Answer briefly.'
c3.key_right = 'What is the country code of the country that speaks !!x!!? Answer briefly.'
c3.filter_key='right'

c4 = Node()
c4.text = ['FILTER',"(Language = 'English')","(Language = 'Dutch')"]
c4.op = 'FILTER'
c4.args=["(Language = 'English')","(Language = 'Dutch')"]

c5 = Node()
c5.text = ['PROJECTION','Region']
c5.op = 'PROJECTION'
c5.args=['Region']
c5.filled_question = 'What is the region that speaks !!x!!?'


c3.l=c1
c3.r = c2
c4.l=c3
c5.l=c4

join_query_trees[q] = c5





# NOT PARSED
# =======================================
# 43
# What is the official language spoken in the country whose head of state is Beatrix?
q = 'SELECT T2.Language FROM country AS T1 JOIN countrylanguage AS T2 ON T1.Code = T2.CountryCode WHERE T1.HeadOfState = "Beatrix" AND T2.IsOfficial = "T"'
# ┌───────────────────────────┐                             
# │         PROJECTION        │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │          Language         │                             
# └─────────────┬─────────────┘                                                          
# ┌─────────────┴─────────────┐                             
# │           FILTER          │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │ (HeadOfState = 'Beatrix') │                             
# │ (IsOfficial = CAST('T' AS │                             
# │          BOOLEAN))        │                             
# └─────────────┬─────────────┘                                                          
# ┌─────────────┴─────────────┐                             
# │      COMPARISON_JOIN      │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │           INNER           ├──────────────┐              
# │    (Code = CountryCode)   │              │              
# └─────────────┬─────────────┘              │                                           
# ┌─────────────┴─────────────┐┌─────────────┴─────────────┐
# │          SEQ_SCAN         ││          SEQ_SCAN         │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   ││   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │          country          ││      countrylanguage      │
# └───────────────────────────┘└───────────────────────────┘                             

c1 = Node()
c1.text = ['SEQ_SCAN','country']
c1.op = 'SEQ_SCAN'
c1.args=['country']

c2 = Node()
c2.text = ['SEQ_SCAN','countrylanguage']
c2.op = 'SEQ_SCAN'
c2.args=['countrylanguage']

c3 = Node()
c3.text = ['JOIN','city']
c3.op = 'JOIN'
c3.key_left='What is the country code of !!x!!? Answer briefly.'
c3.key_right = 'What is the country code of the country that speaks !!x!!? Answer briefly.'
c3.filter_key='left'


c4 = Node()
c4.text = ['FILTER',"(HeadOfState = 'Beatrix')","(IsOfficial = CAST('T' AS BOOLEAN))"]
c4.op = 'FILTER'
c4.args=["(HeadOfState = 'Beatrix')","(IsOfficial = CAST('T' AS BOOLEAN))"]

c5 = Node()
c5.text = ['PROJECTION','Language']
c5.op = 'PROJECTION'
c5.args=['Language']
c5.filled_question = 'What is the language of !!x!!?'



c3.l=c1
c3.r = c2
c4.l=c3
c5.l=c4

join_query_trees[q]=c5





# NOT PARSED
# =======================================
# 44
# what state has no rivers
# SELECT state_name FROM state WHERE state_name NOT IN ( SELECT traverse FROM river );
# ┌───────────────────────────┐                             
# │         PROJECTION        │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │         state_name        │                             
# └─────────────┬─────────────┘                                                          
# ┌─────────────┴─────────────┐                             
# │           FILTER          │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │       (NOT SUBQUERY)      │                             
# └─────────────┬─────────────┘                                                          
# ┌─────────────┴─────────────┐                             
# │      COMPARISON_JOIN      │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │            MARK           ├──────────────┐              
# │   (state_name = #[8.0])   │              │              
# └─────────────┬─────────────┘              │                                           
# ┌─────────────┴─────────────┐┌─────────────┴─────────────┐
# │          SEQ_SCAN         ││         PROJECTION        │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   ││   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │           state           ││          traverse         │
# └───────────────────────────┘└─────────────┬─────────────┘                             
#                              ┌─────────────┴─────────────┐
#                              │          SEQ_SCAN         │
#                              │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
#                              │           river           │
#                              └───────────────────────────┘                             






# NOT PARSED
# =======================================
# 45
# how many rivers do not traverse the state with the capital albany
# SELECT COUNT ( river_name ) FROM river WHERE traverse NOT IN ( SELECT state_name FROM state WHERE capital = "albany" );
# ┌───────────────────────────┐                             
# │         PROJECTION        │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │     count(river_name)     │                             
# └─────────────┬─────────────┘                                                          
# ┌─────────────┴─────────────┐                             
# │         AGGREGATE         │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │     count(river_name)     │                             
# └─────────────┬─────────────┘                                                          
# ┌─────────────┴─────────────┐                             
# │           FILTER          │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │       (NOT SUBQUERY)      │                             
# └─────────────┬─────────────┘                                                          
# ┌─────────────┴─────────────┐                             
# │      COMPARISON_JOIN      │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │            MARK           ├──────────────┐              
# │    (traverse = #[8.0])    │              │              
# └─────────────┬─────────────┘              │                                           
# ┌─────────────┴─────────────┐┌─────────────┴─────────────┐
# │          SEQ_SCAN         ││         PROJECTION        │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   ││   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │           river           ││         state_name        │
# └───────────────────────────┘└─────────────┬─────────────┘                             
#                              ┌─────────────┴─────────────┐
#                              │           FILTER          │
#                              │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
#                              │    (capital = 'albany')   │
#                              └─────────────┬─────────────┘                             
#                              ┌─────────────┴─────────────┐
#                              │          SEQ_SCAN         │
#                              │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
#                              │           state           │
#                              └───────────────────────────┘                             








# NOT PARSED
# =======================================
# 46
# What is the number of distinct continents where Chinese is spoken?
q ='SELECT COUNT( DISTINCT Continent) FROM country AS T1 JOIN countrylanguage AS T2 ON T1.Code = T2.CountryCode WHERE T2.Language = "Chinese"'
# ┌───────────────────────────┐                             
# │         PROJECTION        │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │ count(DISTINCT Continent) │                             
# └─────────────┬─────────────┘                                                          
# ┌─────────────┴─────────────┐                             
# │         AGGREGATE         │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │ count(DISTINCT Continent) │                             
# └─────────────┬─────────────┘                                                          
# ┌─────────────┴─────────────┐                             
# │           FILTER          │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │   (Language = 'Chinese')  │                             
# └─────────────┬─────────────┘                                                          
# ┌─────────────┴─────────────┐                             
# │      COMPARISON_JOIN      │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │           INNER           ├──────────────┐              
# │    (Code = CountryCode)   │              │              
# └─────────────┬─────────────┘              │                                           
# ┌─────────────┴─────────────┐┌─────────────┴─────────────┐
# │          SEQ_SCAN         ││          SEQ_SCAN         │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   ││   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │          country          ││      countrylanguage      │
# └───────────────────────────┘└───────────────────────────┘                             

c1 = Node()
c1.text = ['SEQ_SCAN','country']
c1.op = 'SEQ_SCAN'
c1.args=['country']

c2 = Node()
c2.text = ['SEQ_SCAN','countrylanguage']
c2.op = 'SEQ_SCAN'
c2.args=['countrylanguage']

c3 = Node()
c3.text = ['JOIN','city']
c3.op = 'JOIN'
c3.key_left='What is the country code of !!x!!? Answer briefly.'
c3.key_right = 'What is the country code of the country that speaks !!x!!? Answer briefly.'
c3.filter_key='right'

c4 = Node()
c4.text = ['FILTER',"(Language = 'Chinese')"]
c4.op = 'FILTER'
c4.args=["(Language = 'Chinese')"]
c4.filled_question = 'Is !!x!! the same as Chinese?'

c5 = Node()
c5.text = ['AGGREGATE','count(Continent)']
c5.op = 'AGGREGATE'
c5.args=['count(Continent)']
c5.filled_question = 'What is the continent that speaks !!x!!?'

# c6 = Node()
# c6.text = ['PROJECTION','count(Continent)']
# c6.op = 'PROJECTION'
# c6.args=['count(Continent)']

c3.l=c1
c3.r = c2
c4.l=c3
c5.l=c4
#c6.l=c5

join_query_trees[q] = c5






# NOT PARSED
# =======================================
# 47
# Which region is the city Kabul located in?
q = 'SELECT Region FROM country AS T1 JOIN city AS T2 ON T1.Code = T2.CountryCode WHERE T2.Name = "Kabul"'
# ┌───────────────────────────┐                             
# │         PROJECTION        │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │           Region          │                             
# └─────────────┬─────────────┘                                                          
# ┌─────────────┴─────────────┐                             
# │           FILTER          │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │      (Name = 'Kabul')     │                             
# └─────────────┬─────────────┘                                                          
# ┌─────────────┴─────────────┐                             
# │      COMPARISON_JOIN      │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │           INNER           ├──────────────┐              
# │    (Code = CountryCode)   │              │              
# └─────────────┬─────────────┘              │                                           
# ┌─────────────┴─────────────┐┌─────────────┴─────────────┐
# │          SEQ_SCAN         ││          SEQ_SCAN         │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   ││   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │          country          ││            city           │
# └───────────────────────────┘└───────────────────────────┘                             

c1 = Node()
c1.text = ['SEQ_SCAN','country']
c1.op = 'SEQ_SCAN'
c1.args=['country']

c2 = Node()
c2.text = ['SEQ_SCAN','city']
c2.op = 'SEQ_SCAN'
c2.args=['city']

c3 = Node()
c3.text = ['JOIN','city']
c3.op = 'JOIN'
c3.key_left='What is the country code of !!x!!? Answer briefly.'
c3.key_right = 'What is the country code of the country of !!x!!? Answer briefly.'
c3.filter_key='right'

c4 = Node()
c4.text = ['FILTER',"(Name = 'Kabul')"]
c4.op = 'FILTER'
c4.args=["(Name = 'Kabul')"]
c4.filled_question = 'Is !!x!! the same as Kabul?'
c5 = Node()
c5.text = ['PROJECTION','Region']
c5.op = 'PROJECTION'
c5.args=['Region']
c5.filled_question = 'What is the region of !!x!!?'

c3.l=c1
c3.r = c2
c4.l=c3
c5.l=c4


join_query_trees[q] = c5








# NOT PARSED
# =======================================
# 48
# How many official languages does Afghanistan have?
q = 'SELECT COUNT(*) FROM country AS T1 JOIN countrylanguage AS T2 ON T1.Code = T2.CountryCode WHERE T1.Name = "Afghanistan" AND IsOfficial = "T"'
# ┌───────────────────────────┐                             
# │         PROJECTION        │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │        count_star()       │                             
# └─────────────┬─────────────┘                                                          
# ┌─────────────┴─────────────┐                             
# │         AGGREGATE         │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │        count_star()       │                             
# └─────────────┬─────────────┘                                                          
# ┌─────────────┴─────────────┐                             
# │           FILTER          │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │   (Name = 'Afghanistan')  │                             
# │ (IsOfficial = CAST('T' AS │                             
# │          BOOLEAN))        │                             
# └─────────────┬─────────────┘                                                          
# ┌─────────────┴─────────────┐                             
# │      COMPARISON_JOIN      │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │           INNER           ├──────────────┐              
# │    (Code = CountryCode)   │              │              
# └─────────────┬─────────────┘              │                                           
# ┌─────────────┴─────────────┐┌─────────────┴─────────────┐
# │          SEQ_SCAN         ││          SEQ_SCAN         │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   ││   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │          country          ││      countrylanguage      │
# └───────────────────────────┘└───────────────────────────┘                             

c1 = Node()
c1.text = ['SEQ_SCAN','country']
c1.op = 'SEQ_SCAN'
c1.args=['country']

c2 = Node()
c2.text = ['SEQ_SCAN','countrylanguage']
c2.op = 'SEQ_SCAN'
c2.args=['countrylanguage']

c3 = Node()
c3.text = ['JOIN','city']
c3.op = 'JOIN'
c3.key_left='What is the country code of !!x!!? Answer briefly.'
c3.key_right = 'What is the country code of the country that speaks !!x!!? Answer briefly.'
c3.filter_key='left'

c4 = Node()
c4.text = ['FILTER',"(Name = 'Afghanistan')","(IsOfficial = CAST('T' AS BOOLEAN))"]
c4.op = 'FILTER'
c4.args=["(Name = 'Afghanistan')","(IsOfficial = CAST('T' AS BOOLEAN))"]

c5 = Node()
c5.text = ['AGGREGATE','count_star()']
c5.op = 'AGGREGATE'
c5.args=['count_star()']

# c6 = Node()
# c6.text = ['PROJECTION','count_star()']
# c6.op = 'PROJECTION'
# c6.args=['count_star()']

c3.l=c1
c3.r = c2
c4.l=c3
c5.l=c4
#c6.l=c5

join_query_trees[q] = c5




# NOT PARSED
# =======================================
# 49
# which capitals are not major cities
q = 'SELECT t2.capital FROM state AS t2 JOIN city AS t1 ON t2.capital = t1.city_name WHERE t1.population <= 150000;'
# ┌───────────────────────────┐                             
# │         PROJECTION        │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │          capital          │                             
# └─────────────┬─────────────┘                                                          
# ┌─────────────┴─────────────┐                             
# │           FILTER          │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │ (population <= CAST(150000│                             
# │         AS BIGINT))       │                             
# └─────────────┬─────────────┘                                                          
# ┌─────────────┴─────────────┐                             
# │      COMPARISON_JOIN      │                             
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
# │           INNER           ├──────────────┐              
# │   (capital = city_name)   │              │              
# └─────────────┬─────────────┘              │                                           
# ┌─────────────┴─────────────┐┌─────────────┴─────────────┐
# │          SEQ_SCAN         ││          SEQ_SCAN         │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   ││   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │           state           ││            city           │
# └───────────────────────────┘└───────────────────────────┘                             

c1 = Node()
c1.text = ['SEQ_SCAN','state']
c1.op = 'SEQ_SCAN'
c1.args=['state']

c2 = Node()
c2.text = ['SEQ_SCAN','city']
c2.op = 'SEQ_SCAN'
c2.args=['city']

c3 = Node()
c3.text = ['JOIN','city']
c3.op = 'JOIN'
c3.key_left='What is the capital of !!x!!? Answer briefly.'
c3.key_right = 'What is the name of !!x!!? Answer briefly.'
c3.filter_key='right'

c4 = Node()
c4.text = ['FILTER',"(population <= CAST(150000 AS BIGINT))"]
c4.op = 'FILTER'
c4.args=["(population <= CAST(150000 AS BIGINT))"]

c5 = Node()
c5.text = ['PROJECTION','capital']
c5.op = 'PROJECTION'
c5.args=['capital']
c5.filled_question  = 'What is the capital of the state of !!x!!?'


c3.l=c1
c3.r = c2
c4.l=c3
c5.l=c4

join_query_trees[q] = c5


# NOT PARSED
# =======================================

In [ ]:
# here we translate the operations given by duckdb to question with placeholders for values.
# we then augment some questions to improve performance.
question_maps = json.load(open('data/question_maps.json','r'))
augmented_question_maps = augment_questions(question_maps)

# Tk-Instruct

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tk_model_arch = 'allenai/tk-instruct-large-def-pos'
tokenizer = AutoTokenizer.from_pretrained(tk_model_arch)
model = AutoModelForSeq2SeqLM.from_pretrained(tk_model_arch)

model = model.to(device)

In [ ]:
def tk_form(q): return 'Input: '+q+'\nOutput:'

tk_inst = "Definition: Answer the following questions.\n"

tk_few_shots= """
Input: What is human life expectancy in the United States?
Output: Human life expectancy in the United States is 78 years.

Input: Who was president of the United States in 1955?
Output: Dwight D. Eisenhower was president of the United States in 1955.

Input: Which party did he belong to?
Output: He belonged to the Republican Party.

Input: List some continents.
Output: Asia, Africa, Europe, North America, South America, Oceania, Antarctica

Input: Is Paris in France? Answer with Yes or No.
Output: Yes.

Input: Where were the 1992 Olympics held?
Output: The 1992 Olympics were held in Barcelona, Spain.

"""

### Galois

In [ ]:
HF_SPWJ_seq(model = model,
           tokenizer = tokenizer,
           df=final_sample_df,
           instr=tk_inst,
           few_shots=tk_few_shots,
           inst_funct=tk_form,label='Tk-Inst-FS',
           augmented_question_maps=augmented_question_maps,
           query_plan_dict=join_query_trees,
           device=device,
           verbose=True)


### Single Answer

In [ ]:
single_question_answers = run_question(final_sample_df,model,tokenizer,device)
tk_instruct_final_result_df =pd.DataFrame(single_question_answers,columns=['Single Question Answer'])
tk_instruct_final_result_df['Question']=final_sample_df['Question']
tk_instruct_final_result_df['Query Answer'] = final_sample_df['Answer']
tk_instruct_final_result_df['Database'] = final_sample_df['Database']
tk_instruct_final_result_df['Query'] = final_sample_df['Query']
tk_instruct_final_result_df = tk_instruct_final_result_df[['Question', 'Query Answer', 'Database','Query','Single Question Answer']]


In [ ]:
from src.utils import get_cardinality
results = json.load(open('data/results/tk_instruct/Tk-Inst-FS.json','r'))
# print Galois results and check cardinality
get_cardinality(results)

# Flan UL2

In [ ]:
from transformers import T5ForConditionalGeneration, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/flan-ul2")
model = T5ForConditionalGeneration.from_pretrained("google/flan-ul2",
                                                    device_map="auto")                                                                 


In [ ]:
def flan_inst(q): return 'Question: '+q+'\nAnswer:'

flan_few_shots = """Question: What is human life expectancy in the United States?
Answer: Human life expectancy in the United States is 78 years.

Question: Who was president of the United States in 1955?
Answer: Dwight D. Eisenhower was president of the United States in 1955.

Question: Which party did he belong to?
Answer: He belonged to the Republican Party.

Question: List some continents.
Answer: Asia, Africa, Europe, North America, South America, Oceania, Antarctica

Question: Is Paris in France? Answer with Yes or No.
Answer: Yes.

Question: Where were the 1992 Olympics held?
Answer: The 1992 Olympics were held in Barcelona, Spain.

Question: How many squigs are in a bonk?
Answer: Unknown

"""

### Galois

In [ ]:
HF_SPWJ_seq(model = model,
           tokenizer = tokenizer,
           df=final_sample_df,
           instr='',
           few_shots=flan_few_shots,
           inst_funct=flan_inst,label='Flan-UL2-FS',
           augmented_question_maps=augmented_question_maps,
           query_plan_dict=join_query_trees,
           device=device,
           verbose=True)


### Single Question

In [ ]:
single_question_answers = run_question(final_sample_df,model,tokenizer,device)
flan_ul2_final_result_df =pd.DataFrame(single_question_answers,columns=['Single Question Answer'])
flan_ul2_final_result_df['Question']=final_sample_df['Question']
flan_ul2_final_result_df['Query Answer'] = final_sample_df['Answer']
flan_ul2_final_result_df['Database'] = final_sample_df['Database']
flan_ul2_final_result_df['Query'] = final_sample_df['Query']
flan_ul2_final_result_df = flan_ul2_final_result_df[['Question', 'Query Answer', 'Database','Query','Single Question Answer']]


In [ ]:
from src.utils import get_cardinality
results = json.load(open('data/results/flan_ul2/Flan-UL2-FS.json','r'))

# print Galois results and check cardinality
get_cardinality(results)

